## Extract FL Data

In [63]:
import numpy as np
import pandas as pd

In [64]:
train_df = pd.read_csv('../../data_analysis/fd001/fd001-raw_train.csv', sep=' ')
test_df = pd.read_csv('../../data_analysis/fd001/fd001-raw_test.csv', sep=' ')
train_labels_df = pd.read_csv('../../data_analysis/fd001/fd001-training_labels.csv', sep=' ')

In [65]:
test_df = test_df.groupby('ID').last().reset_index()
test_labels_df = pd.read_csv('../../TED/CMAPSSData/RUL_FD001.txt', sep=' ', header = None)

In [66]:
columns = train_df.columns
ms_used = ['RUL']
for i in range(1, 22):
    if i not in [1, 5, 6, 9, 10, 14, 16, 18, 19]:
        ms_used.append('SensorMeasure' + str(i))

In [67]:
train_labels_df[columns] = train_df[columns]
test_labels_df[columns] = test_df[columns]

In [68]:
train_df = train_labels_df.copy()
test_df = test_labels_df.copy()

In [69]:
train_gb = train_df.groupby(['ID'], sort = False)

In [70]:
# Args: List of pcts -> ex: [0.75, 0.2, 0.05]
def split_data(pcts):

    idx_remaining = [i for i in range(1, len(train_gb) + 1)]
    nums = []
    indices_concat = []
    for pct in pcts:
        nums.append(round(pct * len(train_gb)))
    n_workers = len(pcts)

    for i in range(n_workers):
        idx_worker = np.sort(np.random.choice(idx_remaining, nums[i], replace = False))
        indices_concat.append(idx_worker) 
        idx_remaining = np.setdiff1d(idx_remaining, idx_worker)

    for i in range(len(indices_concat)):
        id_list = indices_concat[i]
        df = train_gb.get_group(id_list[0])
        for j in range(1, len(id_list)):
            df = pd.concat([df, train_gb.get_group(id_list[j])])
        df[ms_used].to_csv('./data/x_partition_' + str(i) + '.csv', sep=' ', index_label='id')

split_data([0.23, 0.27, 0.11, 0.39])